In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off
!pip -qqq install faiss-cpu==1.8.0 --progress-bar off
!pip -qqq install langchain-community==0.2.5 --progress-bar off

  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.41.2 requires huggingface-hub<1.0,>=0.23.0, but you have huggingface-hub 0.20.3 which is incompatible.
transformers 4.41.2 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.1.3 requires langchain-core<0.2.0,>=0.1.45, but you have langchain-core 0.2.9 which is incompatible.


In [49]:
import os
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

In [50]:
!mkdir data
!gdown 1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9 -O "data/meta-earnings.pdf"

mkdir: cannot create directory ‘data’: File exists
Downloading...
From: https://drive.google.com/uc?id=1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9
To: /content/data/meta-earnings.pdf
100% 160k/160k [00:00<00:00, 115MB/s]


## Document Parsing & Spliting

In [51]:
from google.colab import userdata

In [85]:
LLAMA_PARSE_API_KEY = userdata.get("LLAMA_PARSE_API_KEY")

instruction = """The provided document is a delivery chalan or invoice.
This form provides detailed chalan or invoice information about the company's performance for a specific quarter.
It includes unaudited invoice statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables.
Try to be precise while answering the questions"""

parser = LlamaParse(
    api_key = userdata.get("LLAMA_PARSE_API_KEY"),
    result_type = "markdown",
    parsing_instruction = instruction,
    max_timeout = 5000,
)

llama_parse_documents = await parser.aload_data("data/meta-earnings.pdf")
# https://arxiv.org/pdf/1411.1784
# data/meta-earnings.pdf

parsed_doc = llama_parse_documents[0]

# Option - 1
# document_path = Path("parsed_document.md")
# with document_path.open("a") as f:
#     f.write(parsed_doc.text)
# loader = UnstructuredMarkdownLoader(document_path)
# loaded_documents = loader.load()

# Option - 2
loaded_documents = Document(page_content = parsed_doc.text,
                            metadata={"source": "A delivery chalan or invoice."})
loaded_documents = [loaded_documents]

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 400)
docs = text_splitter.split_documents(loaded_documents)

Started parsing the file under job_id cac11eca-64ea-45c2-83aa-6f3feec86457


In [95]:
Markdown(parsed_doc.text)

# Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

# First Quarter 2024 Financial Highlights

|In millions, except percentages and per share amounts|Three Months Ended March 31,|2024|2023|% Change|
|---|---|---|---|---|
|Revenue|$36,455|$28,645|27%| |
|Costs and expenses|$22,637|$21,418|6%| |
|Income from operations|$13,818|$7,227|91%| |
|Operating margin| |38%|25%| |
|Provision for income taxes|$1,814|$1,598|14%| |
|Effective tax rate| |13%|22%| |
|Net income|$12,369|$5,709|117%| |
|Diluted earnings per share (EPS)|$4.71|$2.20|114%| |

# First Quarter 2024 Operational and Other Financial Highlights

- Family daily active people (DAP) – DAP was 3.24 billion on average for March 2024, an increase of 7% year-over-year.
- Ad impressions – Ad impressions delivered across our Family of Apps increased by 20% year-over-year.
- Average price per ad – Average price per ad increased by 6% year-over-year.
- Revenue – Total revenue and revenue on a constant currency basis were $36.46 billion and $36.35 billion, respectively, both of which increased by 27% year-over-year.
- Costs and expenses – Total costs and expenses were $22.64 billion, an increase of 6% year-over-year.
- Capital expenditures – Capital expenditures, including principal payments on finance leases, were $6.72 billion.
- Capital return program – Share repurchases were $14.64 billion of our Class A common stock and dividends payments were $1.27 billion.
- Cash, cash equivalents, and marketable securities – Cash, cash equivalents, and marketable securities were $58.12 billion as of March 31, 2024. Free cash flow was $12.53 billion.
- Headcount – Headcount was 69,329 as of March 31, 2024, a decrease of 10% year-over-year.
---
# CFO Outlook Commentary

We expect second quarter 2024 total revenue to be in the range of $36.5-39 billion. Our guidance assumes foreign currency is a 1% headwind to year-over-year total revenue growth, based on current exchange rates.

We expect full-year 2024 total expenses to be in the range of $96-99 billion, updated from our prior outlook of $94-99 billion due to higher infrastructure and legal costs. For Reality Labs, we continue to expect operating losses to increase meaningfully year-over-year due to our ongoing product development efforts and our investments to further scale our ecosystem.

We anticipate our full-year 2024 capital expenditures will be in the range of $35-40 billion, increased from our prior range of $30-37 billion as we continue to accelerate our infrastructure investments to support our artificial intelligence (AI) roadmap. While we are not providing guidance for years beyond 2024, we expect capital expenditures will continue to increase next year as we invest aggressively to support our ambitious AI research and product development efforts.

Absent any changes to our tax landscape, we expect our full-year 2024 tax rate to be in the mid-teens.

In addition, we continue to monitor an active regulatory landscape, including the increasing legal and regulatory headwinds in the EU and the U.S. that could significantly impact our business and our financial results.

Q1 was a good start to the year. We're seeing strong momentum within our Family of Apps and are making important progress on our longer-term AI and Reality Labs initiatives that have the potential to transform the way people interact with our services over the coming years.
---
Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnings press release, financial tables, and slide presentation. Meta uses the investor.fb.com and about.fb.com/news/ websites as well as Mark Zuckerberg's Facebook Page (facebook.com/zuck), Instagram account (instagram.com/zuck), and Threads profile (threads.net/zuck) as means of disclosing material non-public information and for complying with its disclosure obligations under Regulation FD.

Following the call, a replay will be available at the same website. Transcripts of conference calls with publishing equity research analysts held today will also be posted to the investor.fb.com website.

About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the world. Now, Meta is moving beyond 2D screens toward immersive experiences like augmented and virtual reality to help build the next evolution in social technology.

Contacts

Investors:

Investors:
Kenneth Dorell
investor@meta.com / investor.fb.com
Press:

Press:
Ryan Moore
press@meta.com / about.fb.com/news/
---
Forward-Looking Statements

This press release contains forward-looking statements regarding our future business plans and expectations. These forward-looking statements are only predictions and may differ materially from actual results due to a variety of factors including: the impact of macroeconomic conditions on our business and financial results, including as a result of geopolitical events; our ability to retain or increase users and engagement levels; our reliance on advertising revenue; our dependency on data signals and mobile operating systems, networks, and standards that we do not control; changes to the content or application of third-party policies that impact our advertising practices; risks associated with new products and changes to existing products as well as other new business initiatives, including our artificial intelligence initiatives and metaverse efforts; our emphasis on community growth and engagement and the user experience over short-term financial results; maintaining and enhancing our brand and reputation; our ongoing privacy, safety, security, and content review efforts; competition; risks associated with government actions that could restrict access to our products or impair our ability to sell advertising in certain countries; litigation and government inquiries; privacy, legislative, and regulatory concerns or developments; risks associated with acquisitions; security breaches; our ability to manage our scale and geographically-dispersed operations; and market conditions or other factors affecting the payment of dividends. These and other potential risks and uncertainties that could cause actual results to differ from the results predicted are more fully detailed under the caption "Risk Factors" in our Annual Report on Form 10-K filed with the SEC on February 2, 2024, which is available on our Investor Relations website at investor.fb.com and on the SEC website at www.sec.gov. Additional information will also be set forth in our Quarterly Report on Form 10-Q for the quarter ended March 31, 2024. In addition, please note that the date of this press release is April 24, 2024, and any forward-looking statements contained herein are based on assumptions that we believe to be reasonable as of this date. We undertake no obligation to update these statements as a result of new information or future events.

For a discussion of limitations in the measurement of certain of our community metrics, see the section entitled "Limitations of Key Metrics and Other Data" in our most recent quarterly or annual report filed with the SEC.

Non-GAAP Financial Measures

To supplement our condensed consolidated financial statements, which are prepared and presented in accordance with generally accepted accounting principles in the United States (GAAP), we use the following non-GAAP financial measures: revenue excluding foreign exchange effect, advertising revenue excluding foreign exchange effect, and free cash flow. The presentation of these financial measures is not intended to be considered in isolation or as a substitute for, or superior to, financial information prepared and presented in accordance with GAAP. Investors are cautioned that there are material limitations associated with the use of non-GAAP financial measures as an analytical tool. In addition, these measures may be different from non-GAAP financial measures used by other companies, limiting their usefulness for comparison purposes. We compensate for these limitations by providing specific information regarding the GAAP amounts excluded from these non-GAAP financial measures.

We believe these non-GAAP financial measures provide investors with useful supplemental information about the financial performance of our business, enable comparison of financial results between periods where certain items may vary independent of business performance, and allow for greater transparency with respect to key metrics used by management in operating our business.

Our non-GAAP financial measures are adjusted for the following items:

Foreign exchange effect on revenue. We translated revenue for the three months ended March 31, 2024 using the prior year's monthly exchange rates for our settlement or billing currencies other than the U.S. dollar, which we believe is a useful metric that facilitates comparison to our historical performance.

Purchases of property and equipment; Principal payments on finance leases. We subtract both purchases of property and equipment, net of proceeds and principal payments on finance leases in our calculation of free cash flow because we believe that these two items collectively represent the amount of property and equipment we need to procure to support our business, regardless of whether we procure such property or equipment with a finance lease. We believe that this methodology can provide useful supplemental information to help investors better understand underlying trends in our business. Free cash flow is not intended to represent our residual cash flow available for discretionary expenditures.

For more information on our non-GAAP financial measures and a reconciliation of GAAP to non-GAAP measures, please see the "Reconciliation of GAAP to Non-GAAP Results" table in this press release.

4
---
# META PLATFORMS, INC.

# CONDENSED CONSOLIDATED STATEMENTS OF INCOME

(In millions, except per share amounts)

(Unaudited)

| |Three Months Ended March 31, 2024|Three Months Ended March 31, 2023|
|---|---|---|
|Revenue|$36,455|$28,645|
|Costs and expenses:| | |
|Cost of revenue|6,640|6,108|
|Research and development|9,978|9,381|
|Marketing and sales|2,564|3,044|
|General and administrative|3,455|2,885|
|Total costs and expenses|22,637|21,418|
|Income from operations|13,818|7,227|
|Interest and other income, net|365|80|
|Income before provision for income taxes|14,183|7,307|
|Provision for income taxes|1,814|1,598|
|Net income|$12,369|$5,709|
|Earnings per share:| | |
|Basic|$4.86|$2.21|
|Diluted|$4.71|$2.20|
|Weighted-average shares used to compute earnings per share:| | |
|Basic|2,545|2,587|
|Diluted|2,625|2,596|
---
# META PLATFORMS, INC.

# CONDENSED CONSOLIDATED BALANCE SHEETS

| |March 31, 2024|December 31, 2023|
|---|---|---|
|Assets| | |
|Current assets:| | |
|Cash and cash equivalents|$32,307|$41,862|
|Marketable securities|25,813|23,541|
|Accounts receivable, net|13,430|16,169|
|Prepaid expenses and other current assets|3,780|3,793|
|Total current assets|75,330|85,365|
|Non-marketable equity securities|6,218|6,141|
|Property and equipment, net|98,908|96,587|
|Operating lease right-of-use assets|13,555|13,294|
|Goodwill|20,654|20,654|
|Other assets|8,179|7,582|
|Total assets|$222,844|$229,623|
|Liabilities and stockholders' equity| | |
|Current liabilities:| | |
|Accounts payable|$3,785|$4,849|
|Operating lease liabilities, current|1,676|1,623|
|Accrued expenses and other current liabilities|22,640|25,488|
|Total current liabilities|28,101|31,960|
|Operating lease liabilities, non-current|17,570|17,226|
|Long-term debt|18,387|18,385|
|Long-term income taxes|7,795|7,514|
|Other liabilities|1,462|1,370|
|Total liabilities|73,315|76,455|
|Commitments and contingencies| | |
|Stockholders' equity:| | |
|Common stock and additional paid-in capital|75,391|73,253|
|Accumulated other comprehensive loss|(2,655)|(2,155)|
|Retained earnings|76,793|82,070|
|Total stockholders' equity|149,529|153,168|
|Total liabilities and stockholders' equity|$222,844|$229,623|
---
# CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS

| |Three Months Ended March 31, 2024|Three Months Ended March 31, 2023|
|---|---|---|
|Net income|$12,369|$5,709|
|Adjustments to reconcile net income to net cash provided by operating activities:| | |
|Depreciation and amortization|3,374|2,524|
|Share-based compensation|3,562|3,051|
|Deferred income taxes|-456|-620|
|Impairment charges for facilities consolidation, net|240|770|
|Other|-66|-7|
|Changes in assets and liabilities:| | |
|Accounts receivable|2,520|2,546|
|Prepaid expenses and other current assets|100|821|
|Other assets|-94|30|
|Accounts payable|-1,112|-1,104|
|Accrued expenses and other current liabilities|-1,274|94|
|Other liabilities|83|184|
|Net cash provided by operating activities|19,246|13,998|
|Cash flows from investing activities| | |
|Purchases of property and equipment, net|-6,400|-6,823|
|Purchases of marketable debt securities|-6,887|-85|
|Sales and maturities of marketable debt securities|4,625|534|
|Acquisitions of businesses and intangible assets|-72|-444|
|Other investing activities| |75|
|Net cash used in investing activities|-8,734|-6,743|
|Cash flows from financing activities| | |
|Taxes paid related to net share settlement of equity awards|-3,162|-1,009|
|Repurchases of Class A common stock|-15,008|-9,365|
|Dividends payments|-1,273|0|
|Principal payments on finance leases|-315|-264|
|Other financing activities|-9|122|
|Net cash used in financing activities|-19,767|-10,516|
|Effect of exchange rate changes on cash, cash equivalents, and restricted cash|-288|85|
|Net decrease in cash, cash equivalents, and restricted cash|-9,543|-3,176|
|Cash, cash equivalents, and restricted cash at beginning of the period|42,827|15,596|
|Cash, cash equivalents, and restricted cash at end of the period|$33,284|$12,420|

# Reconciliation of cash, cash equivalents, and restricted cash to the condensed consolidated balance sheets

|Cash and cash equivalents|$32,307|$11,551|
|---|---|---|
|Restricted cash, included in prepaid expenses and other current assets|84|224|
|Restricted cash, included in other assets|893|645|
|Total cash, cash equivalents, and restricted cash|$33,284|$12,420|
---
# META PLATFORMS, INC.

# CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS

| |Three Months Ended March 31, 2024|Three Months Ended March 31, 2023|
|---|---|---|
|Supplemental cash flow data| | |
|Cash paid for income taxes, net|$630|$405|
|Cash paid for interest, net of amounts capitalized|$121|$182|
|Non-cash investing and financing activities:| | |
|Property and equipment in accounts payable and accrued expenses and other current liabilities|$4,217|$4,466|
|Acquisition of businesses in accrued expenses and other current liabilities and other liabilities|$116|$263|
---
# Segment Results

We report our financial results for our two reportable segments: Family of Apps (FoA) and Reality Labs (RL). FoA includes Facebook, Instagram, Messenger, WhatsApp, and other services. RL includes our virtual, augmented, and mixed reality related consumer hardware, software, and content.

**Segment Information**
| |(In millions)|(In millions)|
|---|---|
| |Three Months Ended March 31, 2024|Three Months Ended March 31, 2023|
|Revenue:|$35,635|$28,101|
|Advertising| | |
|Other revenue|$380|$205|
|Family of Apps|$36,015|$28,306|
|Reality Labs|$440|$339|
|Total revenue|$36,455|$28,645|
|Income (loss) from operations:|$17,664|$11,219|
|Family of Apps| | |
|Reality Labs|($3,846)|($3,992)|
|Total income from operations|$13,818|$7,227|
---
# Reconciliation of GAAP to Non-GAAP Results

| |Three Months Ended March 31, 2024|Three Months Ended March 31, 2023|
|---|---|---|
|GAAP revenue|$36,455|$28,645|
|Foreign exchange effect on 2024 revenue using 2023 rates|($106)|($106)|
|Revenue excluding foreign exchange effect|$36,349| |
|GAAP revenue year-over-year change %|27%| |
|Revenue excluding foreign exchange effect year-over-year change %|27%| |
|GAAP advertising revenue|$35,635|$28,101|
|Foreign exchange effect on 2024 advertising revenue using 2023 rates|($105)|($105)|
|Advertising revenue excluding foreign exchange effect|$35,530| |
|GAAP advertising revenue year-over-year change %|27%| |
|Advertising revenue excluding foreign exchange effect year-over-year change %|26%| |

|Net cash provided by operating activities|$19,246|$13,998|
|---|---|---|
|Purchases of property and equipment, net|($6,400)|($6,823)|
|Principal payments on finance leases|($315)|($264)|
|Free cash flow|$12,531|$6,911|

In [87]:
# from pathlib import PosixPath
# PosixPath('parsed_document.md')

## Vector Embeddings & Retriver

In [88]:
HF_TOKEN = userdata.get("HF_TOKEN")
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

database = FAISS.from_documents(docs, embeddings)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [89]:
retriever = database.as_retriever(search_kwargs={"k": 5})

## Reranking

In [90]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor, base_retriever = retriever
)

## Q&A Over Document

In [91]:
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
llm = ChatGroq(temperature = 0,
               model_name = "llama3-70b-8192",
               max_tokens = 3024)

In [92]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(template = prompt_template,
                        input_variables = ["context", "question"])

In [93]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = compression_retriever,
    return_source_documents = True,
    chain_type_kwargs = {"prompt" : prompt, "verbose" : False},
)

In [94]:
# query = "Share all the tabular information in details. Share your response in table format."
query = "Share the details about 'First Quarter 2024 Financial Highlights'. Share your response in table format."
response = qa.invoke(str(query))
Markdown(response["result"])

Running pairwise ranking..


Here are the details about 'First Quarter 2024 Financial Highlights' in table format:

| **Category** | **Three Months Ended March 31,** | **2024** | **2023** | **% Change** |
| --- | --- | --- | --- | --- |
| Revenue |  | $36,455 | $28,645 | 27% |
| Costs and expenses |  | $22,637 | $21,418 | 6% |
| Income from operations |  | $13,818 | $7,227 | 91% |
| Operating margin |  | 38% | 25% |  |
| Provision for income taxes |  | $1,814 | $1,598 | 14% |
| Effective tax rate |  | 13% | 22% |  |
| Net income |  | $12,369 | $5,709 | 117% |
| Diluted earnings per share (EPS) |  | $4.71 | $2.20 | 114% |

Additional helpful information:

* Meta's total revenue and revenue on a constant currency basis were $36.46 billion and $36.35 billion, respectively, both of which increased by 27% year-over-year.
* The company's cash, cash equivalents, and marketable securities were $58.12 billion as of March 31, 2024, with a free cash flow of $12.53 billion.

## References

- [Meta Reports First Quarter 2024 Results](https://s21.q4cdn.com/399680738/files/doc_financials/2024/q1/Meta-03-31-2024-Exhibit-99-1_FINAL.pdf)